In [6]:
# 프로젝트 루트를 Python 경로에 추가
import sys
import os
from pathlib import Path

# 현재 노트북의 위치에서 프로젝트 루트 찾기
current_dir = Path.cwd()
if 'examples' in str(current_dir):
    # examples 폴더에서 실행하는 경우
    project_root = current_dir.parent.parent
else:
    # 프로젝트 루트에서 실행하는 경우
    project_root = current_dir

# 프로젝트 루트를 Python 경로에 추가
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 필요한 모듈 가져오기
from typing import List, Dict, Any, Optional
from datetime import datetime
import polars as pl

from quantbt import (
    # 멀티 타임프레임 전략 시스템
    MultiTimeframeTradingStrategy,
    BacktestEngine,
    
    # 기본 모듈들
    SimpleBroker, 
    BacktestConfig,
    UpbitDataProvider,
    
    # 주문 관련
    Order, OrderSide, OrderType,
)

In [7]:
# 프로젝트 루트를 Python 경로에 추가
import sys
import os
from pathlib import Path

# 현재 노트북의 위치에서 프로젝트 루트 찾기
current_dir = Path.cwd()
if 'examples' in str(current_dir):
    # examples 폴더에서 실행하는 경우
    project_root = current_dir.parent.parent
else:
    # 프로젝트 루트에서 실행하는 경우
    project_root = current_dir

# 프로젝트 루트를 Python 경로에 추가
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 필요한 모듈 가져오기
from typing import List, Dict, Any, Optional
from datetime import datetime
import polars as pl

from quantbt import (
    # 멀티 타임프레임 전략 시스템
    MultiTimeframeTradingStrategy,
    BacktestEngine,
    
    # 기본 모듈들
    SimpleBroker, 
    BacktestConfig,
    UpbitDataProvider,
    
    # 주문 관련
    Order, OrderSide, OrderType,
)


class MultiTimeframeSMAStrategy(MultiTimeframeTradingStrategy):
    """
    단순화된 멀티 타임프레임 SMA 전략
    
    15분봉: 단기 SMA 교차로 매매 신호 생성
    1시간봉: 장기 추세 필터
    
    매수 조건:
    - 15분봉: 단기 SMA(10) > 장기 SMA(30)
    - 1시간봉: 현재 가격 > 추세선 SMA(60)
    
    매도 조건:
    - 15분봉: 단기 SMA(10) < 장기 SMA(30)
    """
    
    def __init__(self):
        timeframe_configs = {
            "15m": { "sma_windows": [10, 30] },
            "1h": { "sma_windows": [60] }
        }
        
        super().__init__(
            name="SimplifiedMultiTimeframeSMA",
            timeframe_configs=timeframe_configs,
            primary_timeframe="15m",  # 기본 타임프레임 변경
            position_size_pct=0.95,   # 포지션 크기 증가
            max_positions=1
        )
        
    def _compute_indicators_for_symbol_and_timeframe(
        self, 
        symbol_data: pl.DataFrame, 
        timeframe: str, 
        config: Dict[str, Any]
    ) -> pl.DataFrame:
        """심볼별 + 타임프레임별 지표 계산"""
        
        data = symbol_data.sort("timestamp")
        indicators = []
        
        if timeframe == "15m":
            # 15분봉: 매매 신호용 SMA
            indicators.extend([
                pl.col("close").rolling_mean(config["sma_windows"][0]).alias("sma_short"),
                pl.col("close").rolling_mean(config["sma_windows"][1]).alias("sma_long")
            ])
            
        elif timeframe == "1h":
            # 1시간봉: 추세 필터용 SMA
            indicators.append(
                pl.col("close").rolling_mean(config["sma_windows"][0]).alias("sma_trend")
            )
        
        return data.with_columns(indicators)
    
    def _analyze_multi_timeframe_signal(
        self, 
        data: Dict[str, Optional[Dict[str, Any]]], 
        symbol: str
    ) -> str:
        """단순화된 크로스 타임프레임 신호 분석"""
        
        d15m = data.get("15m", {})
        d1h = data.get("1h", {})
        
        # 필수 지표 값 추출
        price_15m = d15m.get('close')
        sma_short = d15m.get('sma_short')
        sma_long = d15m.get('sma_long')
        
        price_1h = d1h.get('close')
        sma_trend = d1h.get('sma_trend')

        # 필수 지표 중 하나라도 없으면 홀드
        if any(v is None for v in [price_15m, sma_short, sma_long, price_1h, sma_trend]):
            return "HOLD"
            
        has_position = symbol in self.get_current_positions()

        # 매수 조건
        is_uptrend = price_1h > sma_trend
        is_golden_cross = sma_short > sma_long
        
        if is_uptrend and is_golden_cross and not has_position:
            return "BUY"

        # 매도 조건
        is_dead_cross = sma_short < sma_long
        if is_dead_cross and has_position:
            return "SELL"
        
        return "HOLD"
    
    def generate_signals_multi_timeframe(
        self, 
        multi_current_data: Dict[str, Dict[str, Any]]
    ) -> List[Order]:
        """멀티 타임프레임 신호 생성"""
        
        orders = []
        
        data_15m = multi_current_data.get("15m")
        if not data_15m:
            return orders
            
        symbol = data_15m.get('symbol')
        if not symbol:
            return orders
            
        signal = self._analyze_multi_timeframe_signal(multi_current_data, symbol)
        
        if signal == "BUY":
            orders.append(self._create_buy_order(symbol, data_15m))
        elif signal == "SELL":
            sell_order = self._create_sell_order(symbol)
            if sell_order:
                orders.append(sell_order)
        
        return orders
    
    def _create_buy_order(self, symbol: str, data_15m: Dict[str, Any]) -> Order:
        """매수 주문 생성"""
        current_price = data_15m.get('close', 0)
        portfolio_value = self.get_portfolio_value()
        quantity = self.calculate_position_size(symbol, current_price, portfolio_value)
        
        return Order(
            symbol=symbol,
            side=OrderSide.BUY,
            quantity=quantity,
            order_type=OrderType.MARKET
        )
    
    def _create_sell_order(self, symbol: str) -> Optional[Order]:
        """매도 주문 생성"""
        current_positions = self.get_current_positions()
        quantity = current_positions.get(symbol, 0)
        
        if quantity > 0:
            return Order(
                symbol=symbol,
                side=OrderSide.SELL,
                quantity=quantity,
                order_type=OrderType.MARKET
            )
        return None

In [8]:
# 1. 업비트 데이터 프로바이더
print("🔄 데이터 프로바이더 초기화 중...")
upbit_provider = UpbitDataProvider()

# 2. 멀티 타임프레임 백테스팅 설정
config = BacktestConfig(
    symbols=["KRW-BTC"],
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 12, 31),
    timeframe="15m",                 # 🚀 주요 타임프레임 변경
    initial_cash=10_000_000,
    commission_rate=0.001,
    slippage_rate=0.0005,
    save_portfolio_history=True
)

# 3. 멀티 타임프레임 SMA 전략
print("⚡ 멀티 타임프레임 SMA 전략 초기화 중...")
strategy = MultiTimeframeSMAStrategy()

print(f"📊 전략명: {strategy.name}")
print(f"🕐 사용 타임프레임: {strategy.available_timeframes}")
print(f"📈 멀티 타임프레임: {strategy.is_multi_timeframe_strategy}")
print(f"🎯 주요 타임프레임: {strategy.primary_timeframe}")

# 4. 브로커 설정
broker = SimpleBroker(
    initial_cash=config.initial_cash,
    commission_rate=config.commission_rate,
    slippage_rate=config.slippage_rate
)

# 5. 멀티 타임프레임 백테스트 엔진
print("🚀 멀티 타임프레임 백테스트 엔진 초기화 중...")
engine = BacktestEngine()
engine.set_strategy(strategy)
engine.set_data_provider(upbit_provider)
engine.set_broker(broker)

🔄 데이터 프로바이더 초기화 중...
⚡ 멀티 타임프레임 SMA 전략 초기화 중...
📊 전략명: SimplifiedMultiTimeframeSMA
🕐 사용 타임프레임: ['15m', '1h']
📈 멀티 타임프레임: True
🎯 주요 타임프레임: 15m
🚀 멀티 타임프레임 백테스트 엔진 초기화 중...


In [9]:
# 6. 백테스팅 실행
print("\n" + "=" * 60)
print("🚀 멀티 타임프레임 백테스팅 실행 중...")
print("=" * 60)
print(f"📊 심볼: {config.symbols}")
print(f"📅 기간: {config.start_date} ~ {config.end_date}")
print(f"⏰ 타임프레임: {strategy.available_timeframes}")
print(f"💰 초기 자본: {config.initial_cash:,}원")
print(f"💸 수수료: {config.commission_rate*100:.1f}%")
print(f"📉 슬리피지: {config.slippage_rate*100:.2f}%")

result = engine.run(config)


🚀 멀티 타임프레임 백테스팅 실행 중...
📊 심볼: ['KRW-BTC']
📅 기간: 2024-01-01 00:00:00 ~ 2024-12-31 00:00:00
⏰ 타임프레임: ['15m', '1h']
💰 초기 자본: 10,000,000원
💸 수수료: 0.1%
📉 슬리피지: 0.05%


멀티 타임프레임 백테스팅 진행: 100%|██████████| 34968/34968 [00:00<00:00]


In [10]:
result.print_summary()

                 BACKTEST RESULTS SUMMARY
Period          : 2024-01-01 ~ 2024-12-31
Initial Capital : $10,000,000
Final Equity    : $12,499,172
Total Return    : 24.99%
Annual Return   : 25.01%
Volatility      : 2.84%
Sharpe Ratio    : 8.80
Calmar Ratio    : 1.01
Sortino Ratio   : 1.28
Max Drawdown    : 24.78%
Total Trades    : 968
Win Rate        : 34.7%
Profit Factor   : 1.14
Execution Time  : 0.32s
